<a href="https://colab.research.google.com/github/emadfrj/Sentimental-Analysis-of-Reviews/blob/main/Web_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scrapting reviews for British Airway

Extracting reviews for British Airways from www.airlinequality.com and saving the information in a CSV file. Capturing various review-related fields such as ratings, aircraft details, and verification status of the flight.

## Using BeautifulSoup library to scrap the data

In [1]:
import requests
from bs4 import BeautifulSoup

## Function to capture review rating fields.
These field consist of three types. Text field, rating star, and the airway is recommend or not.

In [2]:
def get_rating_value(ratings,Variable_name):
    for rating in ratings:
        attribute = rating.find('td', class_="review-rating-header")
        if attribute.text.strip() == Variable_name:
            value = rating.find('td', class_="review-value")
            stars = rating.find('td', class_="review-rating-stars")
            #For recommended
            if Variable_name == "Recommended":
                if value.text.strip() == "yes":
                    return 1
                if value.text.strip() == "no":
                    return 0
            #For text field
            elif value:
                return value.text.strip()
            #For rrating stars
            elif stars:
                fillstars = stars.find_all('span', class_ = 'fill')
                return len(fillstars)

    return ""

## Remove special characters and clean the text

In [3]:
import re
def remove_special_characters(input_string):
    # Use a regular expression to remove non-alphanumeric characters
    cleaned_string = re.sub(r'[^a-zA-Z0-9\s]', '', input_string)
    return cleaned_string

## Get the data and store it in Reviews list

In [4]:
# Url to get 1000 reviews
url = 'https://www.airlinequality.com/airline-reviews/british-airways/?sortby=post_date%3ADesc&pagesize=1000'

# Send an HTTP GET request to the URL
response = requests.get(url)
Reviews = []
# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the article block that contain review
    # the specified classes of the article block
    target_classes = ['comp', 'comp_reviews-airline', 'querylist', 'position-content']
    article_block = soup.find('article', class_=target_classes)

    if article_block:
        # Find all child articles within the article block(list of reviews)
        review_articles = article_block.find_all('article')

        # for each review
        for review_article in review_articles:
            #an empty row to add to Reviews list
            Review = {}
            #Getting overall Rating
            overallRatingDiv = review_article.find('div', class_="rating-10")
            overallRating = eval(overallRatingDiv.text.strip())
            Review['overallRating'] = float(overallRating)
            #Getting Text of review
            textdiv = review_article.find('div', class_="text_content")
            Review['Text'] = remove_special_characters(textdiv.text.strip())
            #Trip Verification Status
            verified = textdiv.find('em')
            if verified.text.strip() == "Trip Verified":
                Review['verified'] = 1
            else:
                Review['verified'] = 0
            #Other fields
            #Getting rows of a table consisting other fields
            ratings_list = review_article.find('table')
            ratings = ratings_list.find_all('tr')
            # text fields
            Review['Aircraft'] = get_rating_value(ratings,'Aircraft')
            Review['Type Of Traveller'] = get_rating_value(ratings,'Type Of Traveller')
            Review['Seat Type'] = get_rating_value(ratings,'Seat Type')
            Review['Route'] = get_rating_value(ratings,'Route')
            Review['Date Flown'] = get_rating_value(ratings,'Date Flown')
            # star rating fields
            Review['Seat Comfort'] = get_rating_value(ratings,'Seat Comfort')
            Review['Cabin Staff Service'] = get_rating_value(ratings,'Cabin Staff Service')
            Review['Food & Beverages'] = get_rating_value(ratings,'Food & Beverages')
            Review['Inflight Entertainment'] = get_rating_value(ratings,'Inflight Entertainment')
            Review['Ground Service'] = get_rating_value(ratings,'Ground Service')
            Review['Wifi & Connectivity'] = get_rating_value(ratings,'Wifi & Connectivity')
            Review['Value For Money'] = get_rating_value(ratings,'Value For Money')
            # is recommended field
            Review['Recommended'] = get_rating_value(ratings,'Recommended')
            # Adding the row to the list of Reviews
            Reviews.append(Review)
    else:
        print("Article block not found.")

else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

#print(Reviews)

## Storing the data to CSV file

In [5]:
import csv

# CSV file path
csv_file_path = 'BritishAirwayReviews.csv'

# Writing to CSV file
with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = list(Reviews[0].keys())
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write the header
    writer.writeheader()

    # Write the data
    writer.writerows(Reviews)

print(f'Data has been written to {csv_file_path}')

Data has been written to BritishAirwayReviews.csv
